# <center> **Progetto di Open Data Management 2022-2023**

<br>
<div align="center">
    <img src="op_wallpaper.jpg" width="1000" height="563">

### <center> _Andrea Spinelli, Raffaele Terracino, Marco Valenti_
##### <center> 23 Giugno 

***
***

# __Indice__
###  [`1. Traccia`](#)
### [`2. Selezione dei dataset`](#)
##### &emsp;&emsp; [`2.1 Raccolta`](#)
##### &emsp;&emsp; [`2.2 Licenze`](#)
### [`3. Elaborazione dei dataset`](#)
##### &emsp;&emsp; [`3.1 Pulizia`](#)
##### &emsp;&emsp; [`3.2 Pivoting`](#)
##### &emsp;&emsp; [`3.3 Arricchimento`](#)
### [`4. Formattazione dei dataset a 5 stelle`](#)
### [`5. Creazione di un'applicazione`](#)

***
***

# __1 Traccia__

<u> Utilizzando il linguaggio Python, per lo sviluppo del progetto si devono innanzitutto rispettare i seguenti passi: </u>

- _Selezione dati;_
- _Elaborazione dati (data cleaning, definizione struttura omogenea);_
- _Open Linked Data (creazione di uno strato semantico, ontologie, interlinking)._
 
<u> Dopodiché si può passare, opzionalmente, allo sviluppo del servizio tramite: </u> 
- _Creazione di un bot telegram._

<u> Siano i seguenti dataset (forniti in allegato): </u>

__farmacie.csv,__

__parafarmacie.csv,__

__strutture_sanitarie_pubbliche.csv,__

__strutture_sanitarie_private.csv.__

<u> Si vuole realizzare un servizio in ambito “Salute” che fornisca le posizioni e le disponibilità delle strutture mediche nella regione siciliana. </u>

Si suppone che un utente del servizio abbia bisogno di dover andare a fare una visita dermatologica (struttura privata), oppure una visita ad un consultorio (struttura pubblica), tuttavia non conosce la posizione della struttura; quindi, può interrogare il servizio affinché possa trovare in un raggio di 𝑥 metri, data la sua posizione, un elenco (messaggio testuale, posizioni, …) di strutture specializzate nel campo desiderato. 

È possibile inoltre che l’utente, dopo una visita, abbia la prescrizione di alcuni farmaci; pertanto, avrà bisogno della locazione delle farmacie, o parafarmacie, più vicine. Il servizio di tale richiesta funziona allo stesso modo di quello precedente, inoltre si può fornire anche il recapito telefonico per effettuare la prenotazione dei farmaci. 

***

# __2 Selezione dei dataset__

### __2.1 Raccolta__
### __2.2 Licenze__

***

# __3 Elaborazione dei dataset__

### __3.1 Pulizia__
### __3.2 Pivoting__
### __3.3 Arricchimento__

***


# __4 Formattazione dei dataset a 5 stelle__

***

# __5 Creazione di un'applicazione__

***